In [1]:
import numpy as np
# loaddata = np.load('C:\\Users\\zihan\\Desktop\\Files\\Research\\bryankelly_without_missing.npz')
# data = loaddata['processed_data']
# date= loaddata['stockdate']
fama_french_data= np.load('C:\\Users\\zihan\\Desktop\\Files\\Research\\riskfreerate.npz')
fama_french_data= fama_french_data['fama_french_data']
size_accrual_fama= np.load('C:\\Users\\zihan\\Desktop\\Files\\Research\\size_accrual_fama.npz')
size_accrual_fama= size_accrual_fama['size_accrual_fama']
# print (fama_french_data.shape)
r_f= fama_french_data[:,4]
print (size_accrual_fama[0,1:26])
# normalize the interest rate because Fama French is 100%
r_f= r_f/100
date= fama_french_data[:,0]
date_start= np.asscalar(np.argwhere(date==196307))
date_end= np.asscalar(np.argwhere(date==201612))
portfolio_date= size_accrual_fama[:,0]
portfolio_date_start= np.asscalar(np.argwhere(portfolio_date==196307))
portfolio_date_end= np.asscalar(np.argwhere(portfolio_date==201612))
fama_portfolio= size_accrual_fama[portfolio_date_start:portfolio_date_end+1,1:26]/100
print (fama_portfolio.shape)
portfolio= fama_portfolio
portfolio= fama_portfolio-(r_f[date_start:date_end+1,np.newaxis])
fama_factor= fama_french_data[date_start:date_end+1,1:4]/100
print (fama_factor.shape)


[ 0.1854  0.5055  1.1935  0.0716 -0.4881 -1.7162  1.8097 -1.5418  0.9921
  0.0227 -3.1951 -3.4434 -1.91    0.2139 -2.6296 -1.2645 -0.6369 -1.8085
 -1.8777 -2.3964  0.9027  0.0536  0.374   0.5538 -2.1444]
(642, 25)
(642, 3)


In [3]:
T, N= portfolio.shape
K = 11
window= 120
Factor = np.zeros((T-1-window, K))
gamma = -1
for t in range(T-1-window):
    # if t % 20 == 0:
    #     print('Have gone through iteration %d' % (t))
    portfolio_t = portfolio[t:(t + window), :]
    portfolio_demean_t = portfolio_t - np.mean(portfolio_t, axis=0, keepdims= True)
    covmat_t= portfolio_demean_t.T.dot(portfolio_demean_t)
    variance_t, loading_t = np.linalg.eig(covmat_t)
    Factor[t, :] = portfolio[t + window, :].dot(loading_t[:, :K])
    # print (Factor.shape)
output = []
for K in range(1, 11):
    mu = np.mean(Factor[:, :K], axis=0)
    covmat_factor = np.cov(Factor[:, :K].T)
    if K == 1:
        output.append(abs(mu) * covmat_factor ** (-1 / 2))
    else:
        output.append(np.sqrt(np.dot(np.dot(mu.T, np.linalg.inv(covmat_factor)), mu)))
twodecimals = ["&%.2f" % np.real(v) for v in output]
print(''.join(twodecimals))

&0.08&0.12&0.12&0.12&0.13&0.15&0.16&0.16&0.17&0.17


In [75]:

# Rolling window approach
import scipy.linalg as la
K = 11
window= 120
Factor = np.zeros((T-1-window, K))
gamma = 100
for t in range(T-1-window):
    # if t % 20 == 0:
    #     print('Have gone through iteration %d' % (t))
    portfolio_t = portfolio[t:(t + window), :]
    covmat_t = portfolio_t.T.dot(portfolio_t) 
    portfolio_mean_t = np.sum(portfolio_t, axis=0, keepdims= True)
    # print (portfolio_mean_t.shape)
    covmat_t = covmat_t + portfolio_mean_t.T.dot(portfolio_mean_t) / window * gamma
    variance_t, loading_t = np.linalg.eig(covmat_t)
    Factor[t, :] = portfolio[t + window, :].dot(loading_t[:, :K])
output = []
for i in range(1, 11):
    mu = np.mean(Factor[:, :i], axis=0)
    covmat_factor = np.cov(Factor[:, :i].T)
    if i == 1: 
        output.append(abs(mu) * covmat_factor ** (-1 / 2))
    else:
        output.append(np.sqrt(np.dot(np.dot(mu.T, np.linalg.inv(covmat_factor)), mu)))
twodecimals = ["&%.2f" % np.real(v) for v in output]
print(''.join(twodecimals))
# mu_fama= np.mean(fama_factor, axis= 0)
# # print (mu_fama)
# cov_fama= np.cov(fama_factor.T)
# # print (cov_fama)
# print (np.sqrt(np.dot(mu_fama.T, np.dot(np.linalg.inv(cov_fama),mu_fama))))
# print (len(np.where(Factor==0)))


&0.08&0.12&0.12&0.12&0.13&0.15&0.16&0.16&0.17&0.17


In [70]:
covmat = portfolio.T.dot(portfolio) 
portfolio_mean= np.sum(portfolio, axis= 0, keepdims=True)
# print (portfolio_mean.shape)
gamma= 100
covmat= covmat+ portfolio_mean.T.dot(portfolio_mean)/(T-1)*gamma
variance, loading = np.linalg.eig(covmat)
loading= loading*np.sqrt(N)
output= []
loading_chosen = loading[:, :11]
Factor = portfolio.dot(loading_chosen)/N
for i in range(1, 11): *
    mu = np.mean(Factor[:,:i], axis=0)
    covmat_factor = np.cov(Factor[:,:i].T)
    if i==1: 
        output.append (abs(mu)*covmat_factor**(-1/2))
    else:
        output.append(np.sqrt(np.dot(np.dot(mu, np.linalg.inv(covmat_factor)), mu.T)))
twodecimals = ["&%.2f" % np.real(v) for v in output]
print (''.join(twodecimals))

&0.13&0.13&0.33&0.33&0.33&0.33&0.33&0.33&0.34&0.35


In [60]:
# Get out of sample performance for Fama-French Factors
import scipy.linalg as la
K = 3
window= 120
Factor = np.zeros((T-1-window, K))
for t in range(T-1-window):
    # if t % 20 == 0:
    #     print('Have gone through iteration %d' % (t))
    portfolio_t = portfolio[t:(t + window), :]
    # print (portfolio_t.shape)
    # portfolio_demean_t = portfolio_t - np.mean(portfolio_t, axis=0, keepdims= True)
    # covmat_t= portfolio_demean_t.T.dot(portfolio_demean_t)
    # variance_t, loading_t = la.eigh(covmat_t, eigvals=(0, K))
    factor_t= fama_factor[t:(t+window),:]
    loading_t= portfolio_t.T.dot(factor_t.dot(np.linalg.inv(factor_t.T.dot(factor_t))))
    Factor[t, :] = portfolio[t + window, :].dot(loading_t)
    # print (Factor.shape)
output = []
mu = np.mean(Factor[:, :K], axis=0)
covmat_factor = np.cov(Factor[:, :K].T)
output.append(np.sqrt(np.dot(np.dot(mu.T, np.linalg.inv(covmat_factor)), mu)))
twodecimals = ["&%.2f" % np.real(v) for v in output]
print(''.join(twodecimals))

&0.22
